## Analysing H1B Visa data Trends

H1B visa is a nonimmigrant visa issued to gradaute level applicants allowing them to work in the United States. The employer sponsors the H1B visa for workers with theoretical or technical expertise in specialized fields such as in IT, finance, accounting etc. An interesting fact about immigrant workers is that about 52 percent of new Silicon valley companies were founded by such workers during 1995 and 2005. Some famous CEOs like Indira Nooyi (Pepsico), Elon Musk (Tesla), Sundar Pichai (Google),Satya Nadella (Microsoft) once arrived to the US on a H1B visa.\
**Motivation**: Our team consists of five international graduate students, in the future we will be applying for H1B visa. The visa application process seems very long, complicated and uncertain. So we decided to understand this process and use Machine learning algorithms to predict the acceptance rate and trends of H1B visa.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install autocorrect
import string
import pandas as pd
import numpy as np
import nltk,warnings
import clean_wage as cw
import baseline as blc
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statistics import mean
from autocorrect import Speller 
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import OneHotEncoder #ONE HOT ENCODING
from sklearn.ensemble import RandomForestClassifier #Build model - Random Forest Classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import gensim.downloader as api
nltk.download('wordnet')
nltk.download('words')

### Data 
The data used in the project has been collected from <a href="https://www.foreignlaborcert.doleta.gov/performancedata.cfm">the Office of Foreign Labor Certification (OFLC).</a>The Data provides insight into each petition with information such as the Job title, Wage, Employer, Worksite location etc. To get the dataset click on the above link-> click on Disclosure data -> scroll down to H1B data.\


In [0]:
file2015=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY15_Q4.csv',encoding='latin-1', low_memory=False)
file2015.rename(columns = {'H-1B_DEPENDENT':'H1B_DEPENDENT','SOC_NAME':'SOC_TITLE','WAGE_RATE_OF_PAY':'WAGE_RATE_OF_PAY_FROM'}, inplace = True)
df2015=file2015[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]
df2015['WAGE_RATE_OF_PAY_FROM']=df2015['WAGE_RATE_OF_PAY_FROM'].str.split('-').str[0]

In [0]:
file2016=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY16.csv',encoding='latin-1', low_memory=False)
file2016.rename(columns = {'H-1B_DEPENDENT':'H1B_DEPENDENT','SOC_NAME':'SOC_TITLE'}, inplace = True)
df2016=file2016[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]

In [0]:
file2017=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY17.csv',encoding='latin-1', low_memory=False)
file2017.rename(columns = {'SOC_NAME':'SOC_TITLE'}, inplace = True)
df2017=file2017[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]

In [0]:
file2018=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY2018_EOY.csv',encoding='latin-1', low_memory=False)
file2018.rename(columns = {'SOC_NAME':'SOC_TITLE'}, inplace = True)
df2018=file2018[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]

In [0]:
file2019=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY2019.csv',encoding='latin-1', low_memory=False)
file2019.rename(columns = {'H-1B_DEPENDENT':'H1B_DEPENDENT','WAGE_RATE_OF_PAY_FROM_1':'WAGE_RATE_OF_PAY_FROM','WAGE_UNIT_OF_PAY_1':'WAGE_UNIT_OF_PAY',\
                           'WORKSITE_CITY_1':'WORKSITE_CITY','WORKSITE_STATE_1':'WORKSITE_STATE'}, inplace = True)
df2019=file2019[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]

In [0]:
cleaned=pd.concat([df2015,df2016,df2017,df2018,df2019])
cleaned.shape

In [0]:
del file2015 # To avoid using the RAM completely
del file2016
del file2017
del file2018
del file2019
del df2019
del df2018
del df2017
del df2016
del df2015

### Exploratory Data Analysis
Before we begin working on our data we need to understand the traits of our data which we accomplish using EDA. We see that we have about 260 columns , not all 260 columns have essential information that contributes to our analysis. Hence we pick out the columns such as case status( Accepted/ Denied) ,Employer, Job title etc.We can look at all the colums and the types of object in each column using cleaned.info(). We also use cleaned['column_name'].value_counts() to find the classes in the column along with the count. For feature engineering we are converting quantitative data to categorical.


####Granularity:
Each row in the dataframe represents an application that was filled by the employer and some values are added during the processing of the application such as DECISION_DATE, CASE_STATUS.

In [0]:
cleaned.dropna(inplace=True)
cleaned.shape

In [0]:
cleaned.head(3)

In [0]:
cleaned['VISA_CLASS'].value_counts()

In [0]:
# Visa class has many categories which are not of use , we require only H1B visa type , hence we drop all records with other visa types
cleaned.drop(labels=cleaned.loc[cleaned['VISA_CLASS']!='H-1B'].index , inplace=True)

In [0]:
cleaned['CASE_STATUS'].value_counts()

In [0]:
#In case status column we can drop withdrawn records and certified-withdrawn records
cleaned.drop(labels=cleaned.loc[cleaned['CASE_STATUS']=='WITHDRAWN'].index , inplace=True)
cleaned.drop(labels=cleaned.loc[cleaned['CASE_STATUS']=='CERTIFIED-WITHDRAWN'].index , inplace=True)
cleaned['CASE_STATUS'].value_counts()

In [0]:
cleaned.info()

In [0]:
cleaned['WAGE_RATE_OF_PAY_FROM'].apply(type).value_counts()

####Temporality
The data is available from 9/9/2019 to 1/1/2015

In [0]:
max_date=cleaned.CASE_SUBMITTED.max()	
min_date=cleaned.CASE_SUBMITTED.min()
print("The date in the dataset ranges from "+str(max_date)+" to",min_date)

In [0]:
#the column wages has a mix of both string and float value types and some record have the symbol '$' which we want to remove
cleaned['WAGES']=cleaned['WAGE_RATE_OF_PAY_FROM'].apply(cw.clean_wages).astype('float')

In [0]:
# the wage information that we have available has different unit of pay
cleaned['WAGE_UNIT_OF_PAY'].value_counts()

In [0]:
# we convert the different units of pay to the type 'Year'
cleaned= cw.clean_wageUnit(np,cleaned)
cleaned.drop(columns=['WAGE_RATE_OF_PAY_FROM','WAGE_UNIT_OF_PAY'],axis=1,inplace=True)

In [0]:
#checking if the WAGES column have outliers
print(cleaned['WAGES'].describe())
sns.boxplot(y=cleaned['WAGES'])
plt.show()

We can see that we have large outliers , we can remove them from the data. 

In [0]:
# Remove the outliers.
lowerBound = 0.001
upperBound = 0.95
result = cleaned.WAGES.quantile([lowerBound, upperBound])
cleaned = cleaned[(result.loc[lowerBound] < cleaned.WAGES.values) &\
                  (cleaned.WAGES.values < result.loc[upperBound])]

print(cleaned['WAGES'].describe())
sns.boxplot(y=cleaned['WAGES'])
plt.show()

In [0]:
#Plot for top 10 companies that have the most applications for H1B visa
Top_Employer=cleaned['EMPLOYER_NAME'].value_counts()[:10]
plt.figure(figsize=[8,8])
ax=sns.barplot(y=Top_Employer.index,x=Top_Employer.values,palette=sns.color_palette('viridis',10))
ax.tick_params(labelsize=12)
for i, v in enumerate(Top_Employer.values): 
    ax.text(.5, i, v,fontsize=15,color='white',weight='bold')
plt.title('Top 10 Companies sponsoring H1B Visa in 2015-2019', fontsize=20)
plt.xlabel("Number of applications filed by the companies",fontsize=15)
plt.show()

In [0]:
cleaned.WORKSITE_STATE.value_counts()

In [0]:
cleaned= cw.clean_states(cleaned)

#### Data Scope
The data available contains applications submitted by Employers from 2015 till 2019. \
Geographical Scope: The WORKSITE_STATE gives information about the state where the primary worksite location. The data set covers all the states of US. 
The Data set also cover different Industries majority of the applications are for IT industry but not limited to IT, it also covers Medical, Pharmacy, Law, Carpentery etc.

In [0]:
cleaned.SOC_TITLE.value_counts()

In [0]:
cleaned.WORKSITE_STATE.value_counts()

####Faithfulness:
Each record in the dataframe represents an application filled by the employer, hence we see a lot of spelling mistakes and some punctutaions like ,(comma) and some numbers in the JOB_TITLE, SOC_TITLE feilds, but there are no inconsistencies and data falsification.

In [0]:
#The Job_title and Soc_title has inconsistencies such as numeric characters and symbols so it has to be cleaned
cleaned['JOB_TITLE']=cleaned.JOB_TITLE.apply(lambda txt: " ".join([cw.remove_num(i) for i in txt.lower().split()]))
cleaned['JOB_TITLE']=cleaned['JOB_TITLE'].str.replace(',', '')

cleaned['SOC_TITLE']=cleaned.SOC_TITLE.apply(lambda txt: " ".join([cw.remove_num(i) for i in txt.lower().split()]))
cleaned['SOC_TITLE']=cleaned['SOC_TITLE'].str.replace(',', '')
cleaned['JOB_TITLE'].value_counts()
warnings.simplefilter("ignore")

In [0]:
#Function to clean data columns, to correct spelling mistakes and convert plural words to singular
lemmatizer = nltk.stem.WordNetLemmatizer()
words = set(nltk.corpus.words.words())
spell = Speller()
def lemmatize_text(text):
     return lemmatizer.lemmatize(text)
def spelling_checker(text):
     return spell(text)

In [0]:
cleaned['SOC_TITLE']=cleaned.SOC_TITLE.apply(lambda txt: " ".join([lemmatize_text(i) for i in txt.lower().split()]))
cleaned['SOC_TITLE']=cleaned.SOC_TITLE.apply(lambda txt: " ".join([spelling_checker(i) for i in txt.lower().split()]))

The has lot of different subcategories which have less than 15 applications in them which does not show much significance , removing them help us condense the data and take only the significant rows.


In [0]:
cleaned= cw.drop_less_significant(cleaned)
cleaned=cw.cat_to_num(cleaned)

In [0]:
cleaned.loc[(cleaned.CASE_STATUS == "CERTIFIED"),"CASE_STATUS"] = 1
cleaned.loc[(cleaned.CASE_STATUS == "DENIED"),"CASE_STATUS"] = 0
cleaned.loc[(cleaned.FULL_TIME_POSITION == 'Y'),"FULL_TIME_POSITION"] = 1
cleaned.loc[(cleaned.FULL_TIME_POSITION == 'N'),"FULL_TIME_POSITION"] = 0

In [0]:
data_scnt,data_anlst,data_eng,mach_learn=cw.data_jobs(cleaned)

## Are you a international student? Are you looking for jobs in Data Science domain?
The below visualization shows median salary for four data science related jobs and the number of visa applications that are submitted for each job. This gives us insights about the jobs available for international students/workers about the job market for Data Science domain. We get to know the number of jobs available for each caegory as well as the median wage for each category. 

In [0]:
f,ax=plt.subplots(figsize=(8,6))
bplot1=plt.boxplot([data_scnt[data_scnt['WAGES']<200000].WAGES,data_anlst[data_anlst['WAGES']<200000].WAGES,data_eng[data_eng['WAGES']<200000].WAGES,mach_learn[mach_learn['WAGES']<200000].WAGES],patch_artist="True")
ax.set_xticklabels(['Data Scientists','Data Analysts','Data Engineer','Machine Learning'],fontsize=15)
ax.set_title('Salary Distribution for jobs in Data Science field in 2019', fontsize=15)
ax.tick_params(labelsize=10)
colors = ['blue','orange', 'green', 'red'] 
for patch, color in zip(bplot1['boxes'], colors): 
  patch.set_facecolor(color)
plt.show()
datajobs=cw.data_concat(pd,data_scnt,data_anlst,data_eng,mach_learn)
ax2=sns.countplot(x="data", data=datajobs)
ax2.set_title('Number of petitions for each jobs in Data Science field in 2019', fontsize=15)
warnings.simplefilter("ignore")


### **Baseline classifier**


The baseline classifier is done with a basic model. In this case we are taking the mean of the labels ('certified' and 'denied' for H1B visa approvals). It will give us the base accuracy to which we will compare our classifier's accuracy. Our classifier should have a better accuracy than the baseline classifier accuracy.

In [0]:
# This step assigns a binary class label (0 or 1) to each label for H1B visa approval. 
def create_class_labels(processed_data):
    y = processed_data.to_numpy().astype(int)
    return y
X = cleaned['CASE_STATUS'].to_numpy()
y = create_class_labels(cleaned['CASE_STATUS']) # Groundtruth labels for the dataset
counts = cleaned['CASE_STATUS'].value_counts()
print(counts)
print('proportion: ', counts[1]/counts[0], ': 1')

In [0]:
def compute_accuracy(validation, predicted):
    comp = prediction == validation 
    match_counts = np.count_nonzero(comp == True) 
    clasifier_accuracy = match_counts/len(validation)
    return clasifier_accuracy  
def compute_AUC(y, prediction):
    auc = None
    try:
        auc = roc_auc_score(y, prediction)
    except ValueError:
        pass
    return auc

In [0]:
accuracies = []
AUCs = []
kf = sklearn.model_selection.KFold(n_splits=4, random_state=1, shuffle=True)# Testing with K-folds 
for train_idx, test_idx in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx] 
    prediction = blc.run_clasifier(X_train, y_train, X_test, np)
    fold_accuracy = compute_accuracy(y_test, prediction)
    fold_AUC = compute_AUC(y_test, prediction)
    accuracies.append(fold_accuracy)
    if fold_AUC != None: AUCs.append(fold_AUC)
baseline_clasifier_accuracy = mean(accuracies)
print('Baseline accuracy (K-fold): ', baseline_clasifier_accuracy)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)# Testing with regular split
prediction = blc.run_clasifier(X_train, y_train, X_test, np)
split_accuracy = compute_accuracy(y_test, prediction)
split_AUC = compute_AUC(y_test, prediction)
print('Baseline accuracy (split): ', split_accuracy)  
print('AUC K-fold: ', mean(AUCs))
print('AUC split (80-20): ', split_AUC)

The accuracy results of the baseline classifier is 0.99. This result is due to the highly imbalanced data, where there are 624682 CERTIFIED applications and 5158 DENIED applications. The proportion of it is 121.10934470725087 to 1. Therefore, a performance measure based on the accuracy is not a good one. A better performance measure in imbalanced data is the Area under the ROC Curve (AUC). It meassures the likelihood that given two random points (one from the positive and one from the negative class) the classifier will rank the point from the positive class higher than the one from the negative one.

**Features considered for the classifier**

In [0]:
Dataset = cleaned[["CASE_STATUS", "FULL_TIME_POSITION","SOC_TITLE", "WORKSITE_STATE", "WAGES", "H1B_DEPENDENT"]]
Dataset.head()
Dataset.reset_index(drop = True,inplace = True)
Dataset

CONVERTING CATEGORICAL COLUMNS TO NUMERIC COLUMNS

In [0]:
def remove_punctuation(value):
    result = ""
    for c in value:
        if c not in string.punctuation:
            result += c
        else:
            result +=" "
    return result
Dataset["SOC_TITLE"] = Dataset["SOC_TITLE"].apply(lambda x : remove_punctuation(x))


If we have more number of unique values in the categorical columns we can use word_vectors to convert them into numeric columns as one-hot encoding fails to convert.

In [0]:
word_vectors_1 = api.load("glove-wiki-gigaword-100")

In [0]:
df = pd.DataFrame(columns = ["job_position"])
df

In [0]:
# converting job titles to numeric values
def grouping(position):
  # print(position)
  z = np.zeros(100)
  x = position.split()
  count = 0
  for i in x:
    if i in word_vectors_1:
      z += word_vectors_1[i]
      count+=1
  if count>0:
    return z/count
  else:
    return z
df["job_position"] = Dataset["SOC_TITLE"].apply(lambda text : grouping(text))     

In [0]:
 Dataset.drop(columns=["SOC_TITLE"],axis=1,inplace=True)
Dataset.columns
Dataset.reset_index(drop = True, inplace = True)

In [0]:
# one hot encoding to convert states column to numeric values using one hot encoding
Encoding = OneHotEncoder(handle_unknown='ignore',sparse = True)
Encoding_df = pd.DataFrame(Encoding.fit_transform(Dataset[["WORKSITE_STATE"]]).toarray())
Dataset = Dataset.join(Encoding_df)

In [0]:
Dataset.drop(columns=['WORKSITE_STATE'],axis=1,inplace=True)
word_vectors = df[["job_position"]].values
word_vectors.shape

In [0]:
temp = np.vstack(word_vectors.tolist())
temp.shape

In [0]:
# Y represents the output labels
Y = Dataset["CASE_STATUS"].astype(float)
Dataset.drop(columns = ["CASE_STATUS"],inplace = True)

In [0]:
# all_features represent the input labels
other_features = Dataset.iloc[:].values
all_features = np.hstack([temp, other_features])

In [0]:
#Dependent and independent variables
X = all_features

In [0]:
#Split the date into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=20)

In [0]:
#Build model - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

#model_clf = RandomForestClassifier(n_estimators=10, random_state=30)
model_clf = RandomForestClassifier(n_jobs=2,random_state=0)

In [0]:
#train the model
model_clf.fit(X_train,y_train)

In [0]:
#test the model (predict with our test data)
prediction_test = model_clf.predict(X_test)
prediction_test

In [0]:
#compare with original value, Y_test
from sklearn import metrics
print("Accuracy = ", metrics.accuracy_score(y_test, prediction_test))